Notebook is created for dataset : https://www.kaggle.com/datasets/dev523/leaf-disease-detection-dataset/data

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_dir='/kaggle/input/leaf-disease-detection-dataset/dataset/train'
test_dir='/kaggle/input/leaf-disease-detection-dataset/dataset/test'
for d in [train_dir, test_dir]:
    filepaths = []
    labels=[] 
    classlist=sorted(os.listdir(d))
    print(classlist)
    for klass in classlist:
        label=klass.split('__')[1]
        classpath=os.path.join(d, klass)
        flist=sorted(os.listdir(classpath))

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [8]:

for d in [train_dir, test_dir]:
    filepaths = []
    labels=[] 
    classlist=sorted(os.listdir(d))
    for klass in classlist:
        label=klass.split('___')[1]
        classpath=os.path.join(d, klass)
        flist=sorted(os.listdir(classpath))
        for f in flist:
            fpath=os.path.join(classpath,f)
            filepaths.append(fpath)            
            labels.append(label)
    Fseries=pd.Series(filepaths, name='filepaths')
    Lseries=pd.Series(labels, name='labels')        
    if d == train_dir:
        df=pd.concat([Fseries, Lseries], axis=1)
    else:
        test_df=pd.concat([Fseries, Lseries], axis=1)
#train_df, test_df=train_test_split(df, train_size=.9, shuffle=True, random_state=123, stratify=df['labels'])   
print('train_df lenght: ', len(df), '  test_df length: ', len(test_df))
# get the number of classes and the images count for each class in train_df
classes=sorted(list(df['labels'].unique()))
class_count = len(classes)
print('The number of classes in the dataset is: ', class_count)
groups=df.groupby('labels')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
countlist=[]
classlist=[]
for label in sorted(list(df['labels'].unique())):
    group=groups.get_group(label)
    countlist.append(len(group))
    classlist.append(label)
    print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

# get the classes with the minimum and maximum number of train images
max_value=np.max(countlist)
max_index=countlist.index(max_value)
max_class=classlist[max_index]
min_value=np.min(countlist)
min_index=countlist.index(min_value)
min_class=classlist[min_index]
print(max_class, ' has the most images= ',max_value, ' ', min_class, ' has the least images= ', min_value)
# lets get the average height and width of a sample of the train images
ht=0
wt=0
# select 100 random samples of train_df
train_df_sample=df.sample(n=100, random_state=123,axis=0)
for i in range (len(train_df_sample)):
    fpath=train_df_sample['filepaths'].iloc[i]
    img=plt.imread(fpath)
    shape=img.shape
    ht += shape[0]
    wt += shape[1]
print('average height= ', ht//100, ' average width= ', wt//100, 'aspect ratio= ', ht/wt)

train_df lenght:  70295   test_df length:  17572
The number of classes in the dataset is:  21
            CLASS               IMAGE COUNT 
          Apple_scab               2016     
        Bacterial_spot             5453     
          Black_rot                3875     
       Cedar_apple_rust            1760     
Cercospora_leaf_spot Gray_leaf_spot     1642     
         Common_rust_              1907     
         Early_blight              3859     
     Esca_(Black_Measles)          1920     
Haunglongbing_(Citrus_greening)     2010     
         Late_blight               3790     
          Leaf_Mold                1882     
Leaf_blight_(Isariopsis_Leaf_Spot)     1722     
         Leaf_scorch               1774     
     Northern_Leaf_Blight          1908     
        Powdery_mildew             3419     
      Septoria_leaf_spot           1745     
Spider_mites Two-spotted_spider_mite     1741     
         Target_Spot               1827     
Tomato_Yellow_Leaf_Curl_Virus      

In [9]:
len(test_df['labels'].str.lower().unique())

21

In [10]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [12]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.1,
        channel_shift_range = 0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical') 

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [17]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(38,activation='softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 batch_normalization_3 (Bat  (None, 254, 254, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_4 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                      

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
history = model.fit_generator(
        train_generator,
        epochs=5,
        steps_per_epoch=500,
       validation_steps = 300,
        validation_data=validation_generator)

/tmp/ipykernel_47/3977019443.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
500/500 [==============================] - 327s 648ms/step - loss: 5.0366 - accuracy: 0.0642 - val_loss: 12.6353 - val_accuracy: 0.0360
Epoch 2/5
500/500 [==============================] - 305s 610ms/step - loss: 3.7582 - accuracy: 0.0538 - val_loss: 3.6169 - val_accuracy: 0.0425
Epoch 3/5
500/500 [==============================] - 298s 595ms/step - loss: 3.5995 - accuracy: 0.0564 - val_loss: 3.6306 - val_accuracy: 0.0305
Epoch 4/5
500/500 [==============================] - 293s 586ms/step - loss: 3.6199 - accuracy: 0.0452 - val_loss: 3.6343 - val_accuracy: 0.0310
Epoch 5/5
500/500 [==============================] - 289s 578ms/step - loss: 3.5884 - accuracy: 0.0474 - val_loss: 22.0806 - val_accuracy: 0.0197


Not enough compute for better training

In [ ]:
def testing(path):
    test_image=image.load_img(path,target_size=(128,128))
    #print(test_image)
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)

    #print(f"Result is --> {result}")
    fresult=np.max(result)
    label2=label[result.argmax()]
    print(f"your leaf disease is --> {label2}")
#testing(input("Enter your image path-: "))
path='../input/leaf-disease-detection-dataset/images_for_test/CornCommonRust2.JPG'
testing(path)